In [1]:
!pip install tweepy gradio wordcloud matplotlib textblob pandas pytrends plotly seaborn vaderSentiment requests feedparser -q
!pip install -U kaleido
!pip install praw

import praw
import tweepy
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from textblob import TextBlob
import gradio as gr
import tempfile
import pandas as pd
import nltk
import datetime
from pytrends.request import TrendReq
import plotly.express as px
import seaborn as sns
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import requests
import feedparser

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# ===== Twitter API Credentials =====
consumer_key = 'sWoK8Zca1Pn38MSpa1CZUShgU'
consumer_secret = 'FTQCrruIkiw10Ggr78mMmGSuIpp8ka5aL86FxIg0gy2SqWnGdr'
access_token = '1923239602196017152-o0S1qy4oEvy0Y3bG3KWzwMY25FI1Ly'
access_token_secret = 'amiQiXNKboVwYmszwpdvbZjI3XMGcpBFW1WRMKI3FWHNN'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAABgE1wEAAAAAVSU3BI0pjab5IVx5bkSaFLOKdVA%3DVu7nsZrN9yRVdOPBTSfxADNb3VfhADCcze2ZzO5nSVFE7dVca5'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
client = tweepy.Client(bearer_token=bearer_token)

vader_analyzer = SentimentIntensityAnalyzer()
pytrends = TrendReq(hl='en-US', tz=330)

# Reddit API Setup
reddit = praw.Reddit(
    client_id='sykN1Y1UzM9woFTmpVqk1g',
    client_secret='1zQjNZFSLSnnDxY78ia-zIqTwhlfrQ',
    user_agent='MyDataFetcher script by /u/your_reddit_username'
)

### Twitter Functions ###

def fetch_tweets(query, max_results=50):
    tweets = []
    try:
        response = client.search_recent_tweets(
            query=query,
            max_results=max_results,
            tweet_fields=["text", "lang"]
        )
        if response.data:
            for tweet in response.data:
                if tweet.lang == "en":
                    tweets.append(tweet.text)
    except Exception as e:
        tweets.append(f"Error fetching tweets: {e}")
    return tweets

def analyze_sentiment(tweets):
    sentiment_data = {'positive': 0, 'neutral': 0, 'negative': 0}
    sentiments = []
    for tweet in tweets:
        analysis = TextBlob(tweet)
        polarity = analysis.sentiment.polarity
        if polarity > 0:
            sentiment = 'positive'
            sentiment_data['positive'] += 1
        elif polarity == 0:
            sentiment = 'neutral'
            sentiment_data['neutral'] += 1
        else:
            sentiment = 'negative'
            sentiment_data['negative'] += 1
        sentiments.append((tweet, sentiment))
    return sentiment_data, sentiments

def plot_sentiment(sentiment_data):
    labels = sentiment_data.keys()
    sizes = sentiment_data.values()
    plt.figure(figsize=(5, 5))
    plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=['green', 'grey', 'red'])
    plt.axis('equal')
    plt.title('Sentiment Distribution')
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
    plt.savefig(temp_file.name)
    plt.close()
    return temp_file.name

def generate_wordcloud(tweets):
    text = ' '.join(tweets)
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('Word Cloud')
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
    plt.savefig(temp_file.name)
    plt.close()
    return temp_file.name

def export_to_csv(tweet_data):
    df = pd.DataFrame(tweet_data, columns=["Tweet", "Sentiment"])
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".csv", mode='w', newline='', encoding='utf-8')
    df.to_csv(temp_file.name, index=False)
    return temp_file.name

def analyze_twitter(query):
    tweets = fetch_tweets(query)
    if not tweets:
        return "No tweets found or API error.", None, None, None, None
    sentiment_data, tweet_data = analyze_sentiment(tweets)
    sentiment_chart = plot_sentiment(sentiment_data)
    wordcloud_chart = generate_wordcloud([t[0] for t in tweet_data])
    csv_path = export_to_csv(tweet_data)
    tweet_samples = '\n\n'.join([f"🗨️ {t[0]}\nSentiment: {t[1]}" for t in tweet_data[:10]])
    return f"Fetched {len(tweets)} tweets.", sentiment_chart, wordcloud_chart, tweet_samples, csv_path

### Google Trends + News + Reddit Functions ###

def fetch_trends(keywords):
    try:
        pytrends.build_payload(keywords, cat=0, timeframe='now 1-H', geo='', gprop='')
        df = pytrends.interest_over_time()
        if not df.empty:
            df = df.drop(columns='isPartial')
            df.reset_index(inplace=True)
            return df
        else:
            return pd.DataFrame()
    except Exception:
        return pd.DataFrame()

def plot_trends(df, keywords):
    fig = px.line(df, x='date', y=keywords, title="Real-Time Google Trends")
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
    fig.write_image(temp_file.name)
    return temp_file.name

def fetch_news_headlines(keyword):
    url = f"https://news.google.com/rss/search?q={keyword}&hl=en-US&gl=US&ceid=US:en"
    feed = feedparser.parse(url)
    headlines = [entry.title for entry in feed.entries[:10]]  # top 10 headlines
    return headlines

def analyze_news_sentiment(headlines):
    sentiment_results = []
    for headline in headlines:
        score = vader_analyzer.polarity_scores(headline)
        sentiment_results.append({
            'headline': headline,
            'neg': score['neg'],
            'neu': score['neu'],
            'pos': score['pos'],
            'compound': score['compound']
        })
    return pd.DataFrame(sentiment_results)

def plot_news_sentiment_distribution(df_sentiment, keyword):
    plt.figure(figsize=(8,4))
    sns.histplot(df_sentiment['compound'], bins=10, kde=True)
    plt.title(f"Sentiment Score Distribution for News on '{keyword}'")
    plt.xlabel('Compound Sentiment Score')
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
    plt.savefig(temp_file.name)
    plt.close()
    return temp_file.name

def fetch_reddit_posts(keyword, limit=10):
    posts = []
    for submission in reddit.subreddit('all').search(keyword, limit=limit):
        posts.append({
            'title': submission.title,
            'score': submission.score,
            'url': submission.url,
            'created_utc': datetime.datetime.utcfromtimestamp(submission.created_utc),
            'num_comments': submission.num_comments,
            'subreddit': submission.subreddit.display_name
        })
    return posts

def analyze_trends_news_reddit(keywords_str):
    import traceback
    keywords = [kw.strip() for kw in keywords_str.split(",") if kw.strip()]
    if not keywords:
        return "Please enter at least one keyword.", None, None, None, None

    try:
        # Google Trends
        trend_data = fetch_trends(keywords)
        if trend_data.empty:
            trend_img = None
            trend_status = "No Google Trends data found."
        else:
            trend_img = plot_trends(trend_data, keywords)
            trend_status = "Google Trends data fetched."
    except Exception as e:
        trend_img = None
        trend_status = f"Google Trends error: {e}\n{traceback.format_exc()}"

    first_kw = keywords[0]

    try:
        # News headlines + sentiment
        headlines = fetch_news_headlines(first_kw)
        if not headlines:
            news_status = "No news headlines found."
            news_sentiment_img = None
            news_df = pd.DataFrame()
        else:
            news_df = analyze_news_sentiment(headlines)
            news_sentiment_img = plot_news_sentiment_distribution(news_df, first_kw)
            news_status = f"Fetched {len(headlines)} news headlines for '{first_kw}'."
    except Exception as e:
        news_status = f"News fetching error: {e}\n{traceback.format_exc()}"
        news_sentiment_img = None
        news_df = pd.DataFrame()

    try:
        # Reddit posts
        reddit_posts = fetch_reddit_posts(first_kw, limit=10)
        if not reddit_posts:
            reddit_status = "No Reddit posts found."
            reddit_str = ""
        else:
            reddit_status = f"Fetched {len(reddit_posts)} Reddit posts."
            reddit_str = '\n\n'.join(
                [f"Title: {p['title']}\nSubreddit: {p['subreddit']}\nScore: {p['score']}\nComments: {p['num_comments']}\nDate: {p['created_utc'].strftime('%Y-%m-%d %H:%M:%S')}\nURL: {p['url']}"
                for p in reddit_posts]
            )
    except Exception as e:
        reddit_status = f"Reddit fetching error: {e}\n{traceback.format_exc()}"
        reddit_str = ""

    summary = f"Trends: {trend_status}\nNews: {news_status}\nReddit: {reddit_status}"
    return summary, trend_img, news_sentiment_img, reddit_str

### Gradio UI ###

with gr.Blocks() as demo:
    gr.Markdown("# Real Time Social Media Analysis Dashboard")
    with gr.Tab("Twitter Sentiment Analysis"):
        with gr.Row():
            twitter_input = gr.Textbox(label="Enter Keyword or Hashtag (without #)", placeholder="e.g. Bitcoin", lines=1)
            twitter_btn = gr.Button("Analyze Twitter")
        twitter_status = gr.Textbox(label="Status / Summary", interactive=False)
        twitter_sentiment_img = gr.Image(label="Sentiment Distribution")
        twitter_wordcloud_img = gr.Image(label="Word Cloud")
        twitter_sample_tweets = gr.Textbox(label="Sample Tweets with Sentiment", lines=10, interactive=False)
        twitter_csv = gr.File(label="Download CSV of Tweets & Sentiment")

        twitter_btn.click(fn=analyze_twitter, inputs=twitter_input,
                          outputs=[twitter_status, twitter_sentiment_img, twitter_wordcloud_img, twitter_sample_tweets, twitter_csv])

    with gr.Tab("Google Trends + News + Reddit"):
        with gr.Row():
            trends_input = gr.Textbox(label="Enter Keywords (comma separated)", placeholder="bitcoin, ethereum, crypto", lines=1)
            trends_btn = gr.Button("Analyze Trends & News & Reddit")
        trends_status = gr.Textbox(label="Status Summary", interactive=False)
        trends_img = gr.Image(label="Google Trends Chart")
        news_sentiment_img = gr.Image(label="News Sentiment Distribution")
        reddit_posts_text = gr.Textbox(label="Reddit Posts", lines=15, interactive=False)

        trends_btn.click(fn=analyze_trends_news_reddit, inputs=trends_input,
                         outputs=[trends_status, trends_img, news_sentiment_img, reddit_posts_text])

demo.launch()


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 3.7 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://26bb868601312d6321.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
